In [1]:
import matplotlib.pyplot as plt
import os.path as osp
import numpy as np
num_gpus = 4
classes = ("Background", "Meat", "Nuts/seeds", "Eggs", "Beans/lentils/peas", "Fruit", "Grain", "Vegetables", "Dairy", "Sauce/Spread", "Soup/Drink")
palette = np.random.randint(0, 255, size=(len(classes), 3))
palette[0, :] = 0
print(len(palette))

11


In [2]:
from mmcv import Config
import mmcv
cfg = Config.fromfile('./mmsegmentation/configs/deeplabv3plus/deeplabv3plus_r101-d8_512x512_40k_foodseg_with_label.py')

/u/hli5/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [35]:
name = "deeplabv3plus_mobilenet_multi_food103_40k_together"

In [36]:
from mmseg.apis import set_random_seed
norm_cfg = dict(type='SyncBN', eps=0.001, requires_grad=True)
cfg.model.backbone=dict(
        type='MobileNetV3',
        arch='large',
        out_indices=(1, 3, 16),
        norm_cfg=norm_cfg)
cfg.model.pretrained = 'open-mmlab://contrib/mobilenet_v3_large'
cfg.model.decode_head.num_classes = len(classes)
cfg.model.auxiliary_head.num_classes = len(classes)
cfg.model.decode_head.in_channels = 960
cfg.model.decode_head.in_index = 2
cfg.model.decode_head.channels = 128
cfg.model.decode_head.c1_in_channels = 16
cfg.model.auxiliary_head.channels = 128
cfg.model.auxiliary_head.in_channels = 960
cfg.model.auxiliary_head.in_index = 2
cfg.data.samples_per_gpu = 4
cfg.data.workers_per_gpu=12


cfg.runner.max_iters = 80000
cfg.log_config = dict(
    interval=100,
    hooks=[
        dict(type='TextLoggerHook', by_epoch=False)
    ]
)
cfg.evaluation.interval = 5000
cfg.checkpoint_config.interval = 5000
cfg.checkpoint_config.save_optimizer = True
# cfg.model.backbone.frozen_stages=1

cfg.seed = 0
set_random_seed(0, deterministic=False)
print(f'Config:\n{cfg.pretty_text}')
config_path = "mmsegmentation/configs/food103configs/" + name + ".py"
with open(config_path, "w") as f:
    f.write(cfg.pretty_text)

Config:
norm_cfg = dict(type='SyncBN', requires_grad=True)
model = dict(
    type='MultiEncoderDecoder',
    pretrained='open-mmlab://contrib/mobilenet_v3_large',
    backbone=dict(
        type='MobileNetV3',
        arch='large',
        out_indices=(1, 3, 16),
        norm_cfg=dict(type='SyncBN', eps=0.001, requires_grad=True)),
    decode_head=dict(
        type='MultiDepthwiseSeparableASPPHead',
        in_channels=960,
        in_index=2,
        channels=128,
        dilations=(1, 12, 24, 36),
        c1_in_channels=16,
        c1_channels=48,
        dropout_ratio=0.1,
        num_classes=11,
        norm_cfg=dict(type='SyncBN', requires_grad=True),
        align_corners=False,
        loss_decode=dict(type='CombinedCrossEntropyLoss', loss_weight=1.0)),
    auxiliary_head=dict(
        type='FCNHead',
        in_channels=960,
        in_index=2,
        channels=128,
        num_convs=1,
        concat_input=False,
        dropout_ratio=0.1,
        num_classes=11,
        norm

In [37]:
import torch
torch.cuda.empty_cache()


In [38]:
config_path

'mmsegmentation/configs/food103configs/deeplabv3plus_mobilenet_multi_food103_40k_together.py'

In [ ]:
!mmsegmentation/tools/dist_train.sh {config_path} {num_gpus}

In [ ]:
import os
work_dir = './work_dirs/' + name
checkpoint_file = os.path.join(work_dir, "latest.pth")
assert os.path.isfile(
    checkpoint_file), '`{}` not exist'.format(checkpoint_file)
checkpoint_file = os.path.abspath(checkpoint_file)
checkpoint_file

In [ ]:
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False
import mmcv
import numpy as np
from mmcv.runner import load_checkpoint
import mmcv.visualization.image as mmcv_image
# fix for colab

def imshow(img, win_name='', wait_time=0): plt.figure(
    figsize=(50, 50)); plt.imshow(img)


mmcv_image.imshow = imshow
from mmsegmentation.mmseg.apis import inference_segmentor, init_segmentor


config_fname = config_path
# build the model from a config file and a checkpoint file
model = init_segmentor(config_fname, checkpoint_file)
model.CLASSES = classes

In [ ]:
import torch
from mmcv.parallel import collate, scatter
from mmseg.datasets.pipelines import Compose
# load and display an image with Matplotlib
from PIL import Image
def show_result_pyplot(model,
                       img,
                       result,
                       palette=None,
                       fig_size=(20, 8),
                       opacity=0.5,
                       title='',
                       block=True,
                       show_legend=False):
    """Visualize the segmentation results on the image.

    Args:
        model (nn.Module): The loaded segmentor.
        img (str or np.ndarray): Image filename or loaded image.
        result (list): The segmentation result.
        palette (list[list[int]]] | None): The palette of segmentation
            map. If None is given, random palette will be generated.
            Default: None
        fig_size (tuple): Figure size of the pyplot figure.
        opacity(float): Opacity of painted segmentation map.
            Default 0.5.
            Must be in (0, 1] range.
        title (str): The title of pyplot figure.
            Default is ''.
        block (bool): Whether to block the pyplot figure.
            Default is True.
    """
    _, axes = plt.subplots(nrows=1, ncols=3, figsize=fig_size)
    original_image = Image.open(img)
    file_id = img.split('/')[-1].split('.')[0] + '.png'
    gt_path = 'FoodSeg103/Images/ann_dir_edited/test/'+file_id
    gt_labels = Image.open(gt_path).convert('RGB')
    gt_labels = np.array(gt_labels, dtype=np.uint8)
    gt_labels_ids = np.unique(gt_labels)
    for i in gt_labels_ids:
        gt_labels = np.where(gt_labels == [i,i,i], palette[int(i)], gt_labels)
    gt_labels = gt_labels/255
    axes[0].imshow(original_image)
    axes[1].imshow(original_image)
    axes[1].imshow(gt_labels, alpha=opacity)
    
    if hasattr(model, 'module'):
        model = model.module
    img = model.show_result(
        img, result, palette=palette, show=False, opacity=opacity)
    uniques_set = set(np.unique(result))
    uniques_set.update(set(gt_labels_ids.flatten()))
    if(show_legend):
        custom_lines = []
        labels = []
        for i in uniques_set:
            labels.append(model.CLASSES[i])
            rgb_color = [v/255 for v in palette[i]]
            custom_lines.append(plt.Line2D([0], [0], color=tuple(rgb_color), lw=4))
        plt.legend(custom_lines, labels)
    axes[2].imshow(mmcv.bgr2rgb(img))
    plt.title(title)
    plt.tight_layout()
    plt.show(block=block)

class LoadImage:
    """A simple pipeline to load image."""

    def __call__(self, results):
        """Call function to load images into results.

        Args:
            results (dict): A result dict contains the file name
                of the image to be read.

        Returns:
            dict: ``results`` will be returned containing loaded image.
        """

        if isinstance(results['img'], str):
            results['filename'] = results['img']
            results['ori_filename'] = results['img']
        else:
            results['filename'] = None
            results['ori_filename'] = None
        img = mmcv.imread(results['img'])
        results['img'] = img
        results['img_shape'] = img.shape
        results['ori_shape'] = img.shape
        return results


def inference_segmentor(model, img):
    """Inference image(s) with the segmentor.

    Args:
        model (nn.Module): The loaded segmentor.
        imgs (str/ndarray or list[str/ndarray]): Either image files or loaded
            images.

    Returns:
        (list[Tensor]): The segmentation result.
    """
    cfg = model.cfg
    device = next(model.parameters()).device  # model device
    # build the data pipeline
    test_pipeline = [LoadImage()] + cfg.data.test.pipeline[1:]
    test_pipeline = Compose(test_pipeline)
    # prepare data
    data = dict(img=img)
    data = test_pipeline(data)
    data = collate([data], samples_per_gpu=1)
    if next(model.parameters()).is_cuda:
        # scatter to specified GPU
        data = scatter(data, [device])[0]
    else:
        data['img_metas'] = [i.data[0] for i in data['img_metas']]
    # forward the model
    with torch.no_grad():
        result, labels = model(return_loss=False, rescale=True, **data)

    return result, labels

In [ ]:
import glob
import pandas as pd
labels_db = pd.read_csv("food103labels.csv",sep=',')
LABELS = list(model.CLASSES)[1:]
count = 0
def get_label(file_path):
    file_name = file_path.split('/')[-1]
    numpy_array = labels_db.loc[labels_db['filename'] == file_name][LABELS].to_numpy().astype('int64')
    return numpy_array
def to_one_hot(preds, classes):
    labels_classification = []
    for i in range(len(preds)):
        if(preds[i] == 1):
            labels_classification.append(classes[i+1])
    return ", ".join(labels_classification)

In [ ]:
# test a single image and show the results
file_list = glob.glob("FoodSeg103/Images/img_dir_edited/test/*.jpg")
import random
for i in range(2):
    img = random.choice(file_list)
    result, pred_labels = inference_segmentor(model, img)
    pred_labels = torch.nn.Sigmoid()(pred_labels)
    pred_labels[pred_labels>=0.5] = 1
    pred_labels[pred_labels<0.5] = 0
    gt_labels = to_one_hot(get_label(img).squeeze(),model.CLASSES)
    title = "Predicted: " + to_one_hot(pred_labels, model.CLASSES) + "\n Ground truth: " + gt_labels
    show_result_pyplot(model, img, result, palette=palette, title=title, show_legend=True)

In [ ]:
save_dir = work_dir + "/result"
save_dir

In [ ]:
np_acc = np.zeros(len(LABELS))
all_predictions = np.zeros(shape=(len(file_list), len(LABELS)))
all_gt = np.zeros(shape=(len(file_list), len(LABELS)))
def get_label(file_path):    
    file_name = file_path.split('/')[-1]
    numpy_array = labels_db.loc[labels_db['filename'] == file_name][LABELS].to_numpy().astype('int64')
    return numpy_array
for file in file_list:
    mask, preds = inference_segmentor(model, file)
    preds = torch.nn.Sigmoid()(preds)
    preds[preds>=0.5] = 1
    preds[preds<0.5] = 0
    preds = preds.cpu().numpy()
    gt_label = get_label(file).squeeze()
    np_acc = np.add(preds == gt_label, np_acc)
    all_predictions[count, :] = preds
    all_gt[count, :] = gt_label
    count+=1
    print(count, end='\r')
np_acc = np_acc/len(file_list)
c = 0
for l in LABELS:
    print(f"Accuracy of {l}: {np_acc[c]}")
    c = c+1
print(np.average(np_acc))


In [ ]:
from mmseg.core.evaluation import mean_ap

mean_average_precision = mean_ap.mAP(all_predictions, all_gt)
print(mean_average_precision)

In [ ]:
!./mmsegmentation/tools/dist_test.sh {config_path} {checkpoint_file} 4 --show-dir {save_dir}